In [33]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state = 156)

fetch_20newsgroups를 불러오면 파이썬의 딕셔너리와 유사한 Bunch 클래스를 반환한다. (거의 동일하다고 보면 된다.)

key값은 column의 이름을 의미하는 것과 동일하다고 볼 수 있다. 

In [34]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])


형식만 다를 뿐이지, 다른 데이터 세트와 비슷한 칼럼 명을 가지고 있다는 것을 볼 수 있다. 

여기에서 'filenames'라는 칼럼은 컴퓨터에 저장된 디렉토리 명이다. 

In [37]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


타겟 클래스의 값과 분포도를 확인했다. 너무 왜곡되지 않은 분포를 가지고 있고, 첫번째 출력에서 value_counts() 뒤에 .sort_index()를 해서 

인덱스가 순차적으로 정돈되어 나올 수 있게 된 것이다. 

In [38]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

보면 단순하게 기사 내용만 있는 것이 아니라, 작성자, 헤더, 푸터 등이 전부 포함되어 있다. 우리의 실습은 기사의 내용만으로 주제를 분류하는 것

따라서 헤더, 푸터 등을 제거해야 한다. 이것은 fetch_20newsgroups에서 remove 파라미터에 튜플 형식으로 ('headers', 'footers', 'quotes')를 적으면 

기사 내용만 추출할 수 있다. 

In [7]:
from sklearn.datasets import fetch_20newsgroups

#subset='train'으로 학습용 데이터만 추출, remove = ('headers','footers', 'quotes')로 내용만 추출 
train_news = fetch_20newsgroups(subset='train', remove = ('headers' , 'footers','quotes'), random_state = 156)

X_train = train_news.data
y_train = train_news.target

#subset = 'test'으로 테스트 데이터만 추출, remove = ('headers','footers', 'quotes')로 내용만 추출 
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state = 156)

X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


In [40]:
print(X_train[0])



What I did NOT get with my drive (CD300i) is the System Install CD you
listed as #1.  Any ideas about how I can get one?  I bought my IIvx 8/120
from Direct Express in Chicago (no complaints at all -- good price & good
service).

BTW, I've heard that the System Install CD can be used to boot the mac;
however, my drive will NOT accept a CD caddy is the machine is off.  How can
you boot with it then?

--Dave



이렇게 불필요한 정보들을 제거하고, 훈련 세트와 테스트 세트를 분리하였다. 

이제 X_train에 들어있는 정보는 기사의 내용만 들어있다는 것을 확인했다.

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vecotrization으로 피처 벡터화 변환 수행 
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

#학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape : ', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape :  (11314, 101631)


Scikit-learn의 CountVectorizer 클래스는 fit(), transform()으로 단지 CountVectorizer만 수행하는 것이 아니라, 데이터 전처리과정까지 포함하므로,

별도의 데이터 전처리를 해주지 않아도 된다. 이렇게 X_train, X_test를 해당 객체를 이용하여 벡터화한다. 

이 때, fit_tranform()메서드는 쓰지 않는 것을 추천한다. 훈련 데이터 세트에서는 사용해도 무관하지만, 

테스트 데이터 세트에서는 훈련 데이터 세트를 통해서 학습을 한 CountVectorizer객체가 X_test에 적용하여 분리를 하는 것이지, 

X_test를 학습하고, 다시 X_test를 예측하는 것이 아니기 떄문이다. 

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.617


벡터화까지 진행을 했다면 착각하지 말아야 하는 것이, 끝난 것이 아니라, 일반 ML 알고리즘에 입력을 할 데이터를 알맞은 형태로 **변환**시킨것이다. 

이렇게 변환을 시켰다면, 기존에 우리가 알고있는 알고리즘 모델을 통해서 예측을 진행하고, 성능을 평가해야 한다.

CountVectorizer를 통해서 알고리즘을 예측한 정확도가 약 61% 정도 나오는 것을 확인할 수 있다. 

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#LogisticRegression을 이용해 학습/ 예측 / 평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.678


CountVectorizer 대신 TF-IDF만 썼는데도 정확도가 꽤 올라가는 것을 확인할 수 있다. 

일반적으로 문서가 많고 문서 내에 텍스트가 많은 텍스트 분석에서 TF-IDF가 좋은 예측 결과를 가져다가 준다.

In [45]:
#stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용.
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range=(1,2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.690


예측 정확도를 높이는 가장 좋은 방법 2가지 

1. 최적의 ML 알고리즘을 선택하는 것

2. 최상의 피처 전처리를 수행하는 것

 좀더 다양한 파라미터들을 설정해볼 것이다.

In [16]:
from sklearn.model_selection import GridSearchCV

#최적 C 값 도출 튜님 수행. CV는 3폴드 세트로 설정.
params = {'C' : [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring = 'accuracy', verbose = 1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter : ', grid_cv_lr.best_params_)

#최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.1min finished


Logistic Regression best C parameter :  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.704


로지스틱 회귀에서 C파라미터라는 것은 로지스틱 회귀 비용함수에서 L2 정규화를 한 경우,

기존의 비용함수 + 회귀 계수들의 합 * ㅅ/2 이다.(여기에서 ㅅ은 람다를 뜻한다.)

이 람다의 역수를 C파라미터라고 하는 것이다. 즉 C가 작을 수록 람다가 커져 규제가 커지는 것이라고 볼 수 있다.

In [21]:
from sklearn.pipeline import Pipeline

#TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성 
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words = 'english', ngram_range = (1,2), max_df=300)),
                    ('lr_clf', LogisticRegression(C = 10))])

Pipeline은 예전에 최적화 뭐 어떤 역할을 하는 클래스라고 알고 있었는데, 간단하게 

2개 이상의 과정을 하나의 객체로 표현하여 한번에 여러가지 process를 수행할 수 있도록 하는 클래스라고 보면된다. 

여기에서는 tf-idf 벡터 추출과 로지스틱 회귀의 학습/ 예측을 동시에 수행한다고 보면된다. 

마찬가지로 fit, predict()를 통해서 작동한다.

이에 더해 한가지 더 좋은점은 벡터화를 하면 희소 행렬등과 같이 효율성이 떨어지는 데이터들을 저장해야하는 경우들이 있거나 한다. 

하지만 파이프라인을 통해서 일련의 과정을 한꺼번에 수행하게 되면, 데이터를 저장하지 않고도 수행가능하므로, 저장공간을 줄일 수 있다.

**Pipeline과 결합할 수 있는 것들**

텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있다. 

ex> 스케일링 또는 벡터 정규화, PCA 등의 변환 작업 + 분류, 회귀 등의 Estimator를 한번에 결합하는 것.

In [22]:
#별도의 TfidfVectorizer 객체의 fit(), transform() 과 LogisticRegression의 fit(), predict()가 필요 없음 
#pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/  예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.703930


Pipeline으로 묶은 Tf-idf와 lr_clf의 fit(), predict()를 하나의 객체인 pipeline의 fit(), predict()로 한꺼번에 수행하는 것이 가능하다.

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
    ('lr_clf', LogisticRegression())
])

#Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될 
#파라미터 / 하이퍼 파라미터 이름과 값을 설정 
params = {'tfidf_vect__ngram_range' : [(1,1) , (1,2), (1,3)],
         'tfidf_vect__max_df' : [100, 300, 700],
         'lr_clf__C' : [1,5,10]}

#GridSearchCV의 생성자에 Estimator가 아닌 Pipeline객체 입력 
grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv = 3, scoring = 'accuracy', verbose= 1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 LogisticRegression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


### Pipeline + GridSearchCV

파이프라인으로 일련의 과정을 하나로 묶은 객체를 GridSearchCV를 통해서 최적의 하이퍼 파라미터를 찾는 것이다. 

대부분은 비슷하지만, 파이프라인으로 묶은 과정들 중에서 어느 파라미터가 어느 과정에 필요한 파라미터인지 자동으로 인식할 수 없고, 

혹은 이름이 겹치는 파라미터가 존재할 수도 있다. 이런 경우를 위해서 파이프라인 객체가 GridSearchCV의 인자로 들어간 경우, 

객체 변수명  + 언더바 * 2 + 파라미터 변수명 -> 이렇게 입력해야 한다. 